In [1]:
import os
import time
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras, feature_column
from sklearn import model_selection, preprocessing
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow_hub as hub
import PIL.Image as Image
import tensorflow_datasets as tfds

from IPython.core.interactiveshell import InteractiveShell


In [2]:
# 配置项
# 这个要放到设置中文之前否则还是小方框
plt.style.use("seaborn")

# 指定默认字体 用来正常显示中文标签
plt.rcParams['font.sans-serif'] = ['SimHei']
# 解决保存图像是负号'-'显示为方块的问题
plt.rcParams['axes.unicode_minus'] = False

# #全部行都能输出
InteractiveShell.ast_node_interactivity = "all"

In [4]:
x = tf.ones((2, 2))

with tf.GradientTape() as t:
    t.watch(x)
    y = tf.reduce_sum(x)
    # z = y^2 = (xi1+xi2+...xij)^2
    z = tf.multiply(y, y)
    print(z)

dz_dx = t.gradient(z, x)
for i in [0, 1]:
  for j in [0, 1]:
    # 对矩阵中的每一个元素求导
    assert dz_dx[i][j].numpy() == 8.0

tf.Tensor(16.0, shape=(), dtype=float32)


In [6]:
# 接下来是z对y求导
x = tf.ones((2, 2))

with tf.GradientTape() as t:
    t.watch(x)
    y = tf.reduce_sum(x)
    z = tf.multiply(y, y)

dz_dy = t.gradient(z, y)
dz_dy

del t

<tf.Tensor: id=83, shape=(), dtype=float32, numpy=8.0>

In [14]:
# 多梯度
x = tf.constant(3.0)
with tf.GradientTape(persistent=True) as t:
    # 这一句必须加上 不加上返回空
    t.watch(x)
    y = x * x
    z = y * y

# z = y^2=x^4  对y求导 z'(y)= 2y 求x求导z'(x)=4x^3
dz_dy = t.gradient(z, y)
dz_dx = t.gradient(z, x)


dz_dy, dz_dx

del t

(<tf.Tensor: id=141, shape=(), dtype=float32, numpy=18.0>,
 <tf.Tensor: id=150, shape=(), dtype=float32, numpy=108.0>)

In [32]:
# 记录控制流
def f(x, y):
  output = 1.0
  for i in range(y):
    if i > 1 and i < 5:
      output = tf.multiply(output, x)
  # 搞这么麻烦 如果y>=5,就返回x^3; y=4 返回 x^2; y=3, 返回 x; y<=2, 返回1   
  return output

def grad(x, y):
    with tf.GradientTape() as t:
        t.watch(x)
        out = f(x, y)
    return t.gradient(out, x)



x = tf.convert_to_tensor(2.0)


assert grad(x, 6).numpy() == 12.0
assert grad(x, 5).numpy() == 12.0
assert grad(x, 4).numpy() == 4.0

In [34]:
# 高阶梯度
x = tf.Variable(1.0)

with tf.GradientTape() as t:
    with tf.GradientTape() as t2:
        y = x * x * x
    dy_dx = t2.gradient(y, x)

dy2_dx2 = t.gradient(dy_dx, x)

assert dy_dx.numpy() == 3.0
assert dy2_dx2.numpy() == 6.0
